# Recipes for writing high performance Julia code


In [1]:
import Pkg

Pkg.activate("Project.toml")

 Activating environment at `~/julia_talk/Project.toml`


# How fast is Julia?

Lets see a comparision with

1. Simple C 
2. Handwritten Julia 
3. Julia builtin
4. Python builtin 
5. Handwritten Python 

In [2]:
data = rand(10_000_000);

In [3]:
C_code =  """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";


In [4]:
using Libdl
const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end


In [5]:
function c_sum(X::Array{Float64})
    ccall(("c_sum", Clib), 
          Float64,
          (Csize_t, Ref{Float64}),
          length(X), X)
end


c_sum (generic function with 1 method)

In [6]:
using PyCall

apy_list = PyCall.array2py(data)
apy_numpy = PyObject(data)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum");
# get the Numpy "sum" function:
numpy_sum = pyimport("numpy").sum;


In [7]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum";


In [8]:
function mysum(data)
    acc = zero(eltype(data))
    for x in data
        acc += x
    end
    return acc
end

mysum (generic function with 1 method)

In [51]:
zero(Int64)

0

In [10]:
using BenchmarkTools

suite = BenchmarkGroup()
suite["Julia handwritten"]       = @benchmarkable mysum($data)
suite["Julia builtin"]           = @benchmarkable sum($data)
suite["C function"]       = @benchmarkable c_sum($data)
suite["Python builtin (list)"]   = @benchmarkable $pysum($apy_list)
suite["Python builtin (numpy)"]  = @benchmarkable $numpy_sum($apy_numpy)
suite["Python handwritten"]      = @benchmarkable $sum_py($apy_list)

# If a cache of tuned parameters already exists, use it, otherwise, tune and cache
# the benchmark parameters. Reusing cached parameters is faster and more reliable
# than re-tuning `suite` every time the file is included.
paramspath = joinpath(@__DIR__, "sum_bench.json")

if isfile(paramspath)
    loadparams!(suite, BenchmarkTools.load(paramspath)[1], :evals);
else
    tune!(suite)
    BenchmarkTools.save(paramspath, params(suite));
end


6-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "Julia builtin" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Julia handwritten" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "C function" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Python builtin (list)" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Python builtin (numpy)" => Benchmark(evals=1, seconds=5.0, samples=10000)
  "Python handwritten" => Benchmark(evals=1, seconds=5.0, samples=10000)

In [11]:
results = run(suite)


6-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "Julia builtin" => Trial(3.702 ms)
  "Julia handwritten" => Trial(10.938 ms)
  "C function" => Trial(10.687 ms)
  "Python builtin (list)" => Trial(45.432 ms)
  "Python builtin (numpy)" => Trial(4.038 ms)
  "Python handwritten" => Trial(225.513 ms)

In [12]:
for (name, trial) in sort(collect(results), by=x->time(x[2]))
    t = time(trial) / 1e6
    println(rpad(name, 25, "."), lpad(string(round(t, digits=2), " ms"), 20, "."))
end


Julia builtin..........................3.7 ms
Python builtin (numpy)................4.04 ms
C function...........................10.69 ms
Julia handwritten....................10.94 ms
Python builtin (list)................45.43 ms
Python handwritten..................225.51 ms


In [13]:
@which sum(data)

sum(a::AbstractArray; dims) in Base at reducedim.jl:722

# Benchmarking

1. Premature optimization is the root of all evil - Knuth

2. Measure and then optimize


### Tools: 

1. BenchmarkTools.jl https://github.com/JuliaCI/BenchmarkTools.jl

2. Profiler https://docs.julialang.org/en/latest/manual/profile/

3. ProfileView.jl https://github.com/timholy/ProfileView.jl


In [41]:
function mysum(data)
    acc = 0.0
    for x in data
        acc += x
    end
    return acc
end

mysum (generic function with 1 method)

In [42]:
@time mysum(data)

  0.019207 seconds (4.58 k allocations: 219.972 KiB)


5.0007290604401445e6

In [53]:
@time mysum(data)

  0.011305 seconds (1 allocation: 16 bytes)


5.0007290604401445e6

In [17]:
using BenchmarkTools

@benchmark mysum($data)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.972 ms (0.00% GC)
  median time:      11.066 ms (0.00% GC)
  mean time:        11.175 ms (0.00% GC)
  maximum time:     14.581 ms (0.00% GC)
  --------------
  samples:          448
  evals/sample:     1

In [18]:
@btime mysum($data)

  10.965 ms (0 allocations: 0 bytes)


5.0007290604401445e6

# Code Optimizations

## Don't use global variables

In [61]:
x = 10

function f1()
    [i + x for i in 1:10^6]
end

function f2(n)
   return [i + n for i in 1:10^6]
end

f2 (generic function with 1 method)

In [20]:
@benchmark f1()

BenchmarkTools.Trial: 
  memory estimate:  38.13 MiB
  allocs estimate:  1998992
  --------------
  minimum time:     25.114 ms (0.00% GC)
  median time:      26.774 ms (4.22% GC)
  mean time:        27.777 ms (3.25% GC)
  maximum time:     52.412 ms (4.03% GC)
  --------------
  samples:          181
  evals/sample:     1

In [21]:
@benchmark f2($x)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     575.683 μs (0.00% GC)
  median time:      720.028 μs (0.00% GC)
  mean time:        809.589 μs (8.62% GC)
  maximum time:     4.361 ms (15.93% GC)
  --------------
  samples:          6170
  evals/sample:     1

# Code Optimizations

## Add @inbounds

In [22]:
function mygemm1!(C, A, B)
   for j in 1:size(C, 2)
    for k in 1:size(A, 2)
      for i in 1:size(C, 1)
        C[i, j] += A[i, k] * B[k, j]
        end 
    end
  end
  return C
end

function mygemm(A, B)
  @assert size(A, 2) == size(B, 1)
  C = zeros(size(A, 1), size(B,2))
  mygemm1!(C, A, B)
  return C
end


function mygemm!(C, A, B)
   for j in 1:size(C, 2)
    for k in 1:size(A, 2)
      for i in 1:size(C, 1)
        @inbounds C[i, j] += A[i, k] * B[k, j]
        end 
    end
  end
  return C
end

function mygemm_inbounds(A, B)
  @assert size(A, 2) == size(B, 1)
  C = zeros(size(A, 1), size(B,2))
  mygemm!(C, A, B)
  return C
end

mygemm_inbounds (generic function with 1 method)

In [23]:
N = 1024
A = rand(N, N);
B = rand(N, N);

In [24]:
mygemm_inbounds(A, B) ≈ mygemm(A, B) 

true

In [25]:
@btime mygemm_inbounds($A, $B)
@btime mygemm($A, $B);

  369.163 ms (2 allocations: 8.00 MiB)
  1.044 s (2 allocations: 8.00 MiB)


## Use static arrays

Static arrays are statically sized arrays. Using them can be very powerful when the vector and matrix sizes are small (<100 elements). Size information is passed on to the compiler which can help it with doing advanced optimizations. 

```
============================================
    Benchmarks for 3×3 Float64 matrices
============================================
Matrix multiplication               -> 5.9x speedup
Matrix multiplication (mutating)    -> 1.8x speedup
Matrix addition                     -> 33.1x speedup
Matrix addition (mutating)          -> 2.5x speedup
Matrix determinant                  -> 112.9x speedup
Matrix inverse                      -> 67.8x speedup
Matrix symmetric eigendecomposition -> 25.0x speedup
Matrix Cholesky decomposition       -> 8.8x speedup
Matrix LU decomposition             -> 6.1x speedup
Matrix QR decomposition             -> 65.0x speedup
```

- `v1 = @SVector rand(10);`
- `m1 = @SMatrix rand(10, 10);`

Repo: https://github.com/JuliaArrays/StaticArrays.jl



# Check for type instablity

Type stability: When all variables in function have only one possible Type.

Note: In julia types are inferred at run time. 

Lets see what and why this is a problem.

# Julia compiler schematic


![image.png](compiler.png)


To view different stages of compilation use

1. @code_lowered - removes all syntactic sugar and adds code
2. @code_typed - adds type annotations to the lowered code
3. @code_llvm - shows LLVM IR
4. @code_native - shows machine code
5. @code_warntype - highlights type instable parts if any

In [26]:
function mymax(init, x)
   s = init # -Infinity
   for i in 1:length(x)
       xi = x[i]
       s = s < xi ? xi : s
   end
   return s
end

r = rand(Int, 1000);


@btime mymax(-Inf32, r);
@btime mymax(typemin(Int), r);

  4.278 μs (1 allocation: 16 bytes)
  520.298 ns (1 allocation: 16 bytes)


In [27]:
typeof(-Inf32), typeof(typemin(Int))

(Float32, Int64)

In [28]:
@code_warntype mymax(-Inf32, r)

Variables
  #self#::Core.Compiler.Const(mymax, false)
  init::Float32
  x::Array{Int64,1}
  s::Union{Float32, Int64}
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  xi::Int64
  @_8::Union{Float32, Int64}

Body::Union{Float32, Int64}
1 ─       (s = init)
│   %2  = Main.length(x)::Int64
│   %3  = (1:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #7 if not %6
2 ┄ %8  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (xi = Base.getindex(x, i))
│   %12 = (s < xi)::Bool
└──       goto #4 if not %12
3 ─       (@_8 = xi)
└──       goto #5
4 ─       (@_8 = s)
5 ┄       (s = @_8)
│         (@_5 = Base.iterate(%3, %10))
│   %19 = (@_5 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #7 if not %20
6 ─       goto #2
7 ┄       return

In [29]:
@code_warntype mymax(typemin(Int), r)

Variables
  #self#::Core.Compiler.Const(mymax, false)
  init::Int64
  x::Array{Int64,1}
  s::Int64
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  xi::Int64
  @_8::Int64

Body::Int64
1 ─       (s = init)
│   %2  = Main.length(x)::Int64
│   %3  = (1:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #7 if not %6
2 ┄ %8  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (xi = Base.getindex(x, i))
│   %12 = (s < xi)::Bool
└──       goto #4 if not %12
3 ─       (@_8 = xi)
└──       goto #5
4 ─       (@_8 = s)
5 ┄       (s = @_8)
│         (@_5 = Base.iterate(%3, %10))
│   %19 = (@_5 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #7 if not %20
6 ─       goto #2
7 ┄       return s


### Note:
- In @code_warntype output if you see anything annotated in red and of types Union and Any, that indicates that your code has type instability.

## Recap: 

Type stability is very important. It can make or break your performance. Especially, when the whole library is not type stable. 

In [65]:
@code_native @time 1+1

	.text
	.file	"@time"
	.globl	julia_time_5301         # -- Begin function julia_time_5301
	.p2align	4, 0x90
	.type	julia_time_5301,@function
julia_time_5301:                        # @julia_time_5301
	.cfi_startproc
# %bb.0:                                # %top
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	andq	$-32, %rsp
	subq	$288, %rsp              # imm = 0x120
	.cfi_offset %rbx, -56
	.cfi_offset %r12, -48
	.cfi_offset %r13, -40
	.cfi_offset %r14, -32
	.cfi_offset %r15, -24
	movq	%rdx, %r14
	vxorps	%xmm0, %xmm0, %xmm0
	vmovups	%ymm0, 80(%rsp)
	vmovaps	%ymm0, 64(%rsp)
	vmovaps	%ymm0, 32(%rsp)
	#APP
	movq	%fs:0, %rcx
	#NO_APP
	movq	%rcx, 24(%rsp)          # 8-byte Spill
	movq	$32, 32(%rsp)
	movq	-15720(%rcx), %rax
	movq	%rax, 40(%rsp)
	leaq	32(%rsp), %rax
	movq	%rax, -15720(%rcx)
	movabsq	$jl_copy_ast, %r15
	movabsq	$140586945132880, %rbx  # imm = 0x7FDCF2ECB950
	mo

# Parallel computing

## SIMD

SIMD = Single Instruction Multiple Data

This is intra-core parallelism

- Single instruction can operate on multiple vector elements at a time
- Modern CPUs have vector registers of sizes 128, 256, 512 bits
- These registers hold 2, 4 or 8 Float64 numbers or 4, 8 or 16 Float32 numbers


![image.png](simd.png)

- To use SIMD, memory access should be continguos.

### Note: 
1. Make sure there are no dependency chains in your code.
2. There should not be any branches in the code (eg: if-else)


## Loading and storing vectors

![image.png](vec_memory.png)


## Vector summation

![image.png](simd1.png)


## Using SIMD

![image.png](simd2.png)


- This is simplified view of SIMD 


In [31]:
function mysum_slow(data)
    acc = zero(eltype(data))
    @inbounds for x in data
        acc += x
    end
    return acc
end



function mysum_simd(data)
    acc = zero(eltype(data))
    @inbounds @simd for x in data
        acc += x
    end
    return acc
end

x = rand(1024);


In [32]:
@btime mysum_slow($x)
@btime mysum_simd($x);

  1.016 μs (0 allocations: 0 bytes)
  60.510 ns (0 allocations: 0 bytes)


In [33]:
x768 = rand(768);
x799 = rand(799);
x800 = rand(800);

In [34]:
@btime mysum_simd($x768)
@btime mysum_simd($x799)
@btime mysum_simd($x800);

  43.334 ns (0 allocations: 0 bytes)
  60.062 ns (0 allocations: 0 bytes)
  49.658 ns (0 allocations: 0 bytes)


## Enter LoopVectorization

- LoopVectorization.jl does SIMD and also does many more optimizations that can help LLVM perform even faster and better optimizations. 
- With LoopVectorization julia code can perform on par with OpenBLAS/MKL code.
- Use this when you do lot of elementwise operations and also makes special functions such as log, trignometric functions on vectors very fast.

Repo: https://github.com/chriselrod/LoopVectorization.jl

In [35]:
using LoopVectorization

function mysumavx(data)
    acc = zero(eltype(data))
    @avx for i in eachindex(data)
        acc += data[i]
    end
    return acc
end

mysumavx (generic function with 1 method)

In [36]:
@btime mysumavx($x768)
@btime mysumavx($x799)
@btime mysumavx($x800);

  31.807 ns (0 allocations: 0 bytes)
  35.955 ns (0 allocations: 0 bytes)
  32.791 ns (0 allocations: 0 bytes)


## Multithreading and Multiprocessing

If you need to perform many independent operations (i.e. mapping a function over an array), parallel computing will produce a large performance boost.


### Multithreading

Threads are lightweight, since all the threads share the same space in memory, and you can launch as many threads as you want. The CPU can simultaneously a number of threads equal to the number of cores. Since the memory is shared you must pay attention to correct synchronisation when multiple threads are accessing the same variable.

- To use multithreading you have to assign the julia REPL with some threads. You can do this by running `export JULIA_NUM_THREADS=12` and then run `julia`. 
- You can check the number of threads assigned in the REPL using `Threads.nthreads()`


In [67]:
Threads.nthreads()

10

In [38]:
function foo()
    for i in 1:10
        sleep(0.1) # some computation that takes 0.1s
    end
end

@time foo()

  1.012384 seconds (85 allocations: 2.750 KiB)


In [39]:
function foo_threaded()
    Threads.@threads for i in 1:10
        sleep(0.1)
    end
end

@btime foo_threaded()

  100.465 ms (104 allocations: 8.81 KiB)


### Composable multithreading

In [40]:
import Base.Threads.@spawn

function slow_func(x)
    sleep(0.005) 
    return x
end

@btime let
    a = @spawn slow_func(2)
    b = @spawn slow_func(4)
    c = @spawn slow_func(42)
    d = @spawn slow_func(12)
    res = fetch(a) .+ fetch(b) .* fetch(c) ./ fetch(d)
end

@btime let
    a = slow_func(2)
    b = slow_func(4)
    c = slow_func(42)
    d = slow_func(12)
    res = a .+ b .* c ./ d
end;


  5.237 ms (46 allocations: 3.64 KiB)
  23.526 ms (20 allocations: 576 bytes)


## Multiprocessing

Here we can spawn number of processes less than or equal to number of cores

In [40]:
using Distributed

In [41]:
addprocs(4)


# addprocs(list of ip addresses of nodes in the cluster)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [42]:
@everywhere function slow_func(x)
    sleep(0.005) 
    return x
end

In [ ]:
@everywhere using SharedArrays

res = SharedArray(zeros(10))

@btime let
@distributed for x in 1:10
    res[x] = slow_func(x)
end
end

In [ ]:
res

### Note:

- Threads are lightweight and you can easily spawn tasks and parallel loops. Multi-threading is recommended if the computations take more than 100 μs.
- Use multi-processing if you are working with a cluster or if your computations take longer than 100 ms. Remember that spawning a process is an expensive operation.


# Where to get help

1. Julia discourse: https://discourse.julialang.org/ - recommended
2. Julia slack:     https://julialang.org/slack/ - for informal chat